## SSCWeb trajectory: 



In [15]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
import numpy as np
import pandas as pd
import os
import scipy
import scipy.constants as constants
from scipy import interpolate
from scipy.interpolate import RegularGridInterpolator
import math
import time
from kamodo import Kamodo, kamodofy
import kamodo
from datetime import datetime,timezone,timedelta

In [17]:
from readers.sscweb_trajectory import SSCWEB_Trajectory

In [18]:
traj=SSCWEB_Trajectory('goes15',(2018,11,20,20,25,0),(2018,11,20,23,59,0))

Obtaining trajectory from SSCWeb for goes15
variable: X unit: km data.size: 71
variable: Y unit: km data.size: 71
variable: Z unit: km data.size: 71
variable: Xvec unit: km data.size: 213


## Use GEOPACK to perform coordinate transformations

In [19]:
from geopack_transform import transform,seconds_from_19700101

In [20]:
# GSE are Satellite data obtained via SSCWEB_trajectory are in GSE.
# GSE is default for coord_in transform()
# transform entire trajectory to GEO 
# using a different time stamp for each vertex along the trajectory:
Xvec_data_GEO=transform(Time=traj.Time,Xvec=traj.variables['Xvec']['data'],coord_out='GEO')

In [21]:
Time=traj.Time

In [22]:
Xvec=traj.Xvec(traj.Time)

In [23]:
# transform a trajectory to GSM
Xvec_GSM=transform(Xvec=Xvec,Time=Time,coord_in='GSE',coord_out='GSM')

In [24]:
Time[0]

datetime.datetime(2018, 11, 20, 20, 27, tzinfo=tzutc())

In [25]:
Xvec[0,:]

array([39662.3330457 , -1607.46417177, 14154.85507733])

In [26]:
Xvec_GSM[:][0]

[39662.333045704676, -479.56823692406124, 14237.76238142179]

In [27]:
# test will show if transformation and reverse transformation result in the same vector
# the default vector is [0.,0.,1.]
# this vector is the same in GEO and GEI, which differ only by a rotation around the Z-axis
# the vector returned after the inverse transformation should differ from 
# the original only by the numerical roundoff error
def test_geopack_convert():
    coords=['GSE','GSM','SM','MAG','GEO','GEI']
    Time=np.array([datetime(2018,11,20,21,25,0,tzinfo=timezone.utc)])

    print(Time)
    xyz_in=np.array([[0.,0.,1.]])
    for coord_in in coords:
        for coord_out in coords:
            if coord_in != coord_out:
                xyz_out=transform(xyz_in,debug=True,Time=Time,coord_in=coord_in,coord_out=coord_out)
                xyz_in2=transform(xyz_out,Time=Time,coord_in=coord_out,coord_out=coord_in)
                
                print('in: ',coord_in,' out: ',coord_out)
                print(xyz_in)  # input position vector
                print(xyz_out) # position vector in new coordinate system
                print(xyz_in2) # this should be the original vector
                

In [28]:
test_geopack_convert()
# debug=True shows the first time stamp, seconds from Jan. 1, 1970 00:00 UT
# and the resulting psi (tilt angle) value from Geopack's recalc().
# The test will show the input and output coordinate systems,
# the start position, transformed position and 
# the position after reversing the transformation,
# which should be nearly identical to the original position.

[datetime.datetime(2018, 11, 20, 21, 25, tzinfo=datetime.timezone.utc)]
2018-11-20 21:25:00+00:00
1542749100.0
psi= -0.2901445844736547
in:  GSE  out:  GSM
[[0. 0. 1.]]
[[0.0, 0.05876639828690128, 0.9982717618125765]]
[[0.0, 0.0, 1.0000000000000002]]
2018-11-20 21:25:00+00:00
1542749100.0
psi= -0.2901445844736547
in:  GSE  out:  SM
[[0. 0. 1.]]
[[0.2855963363092814, 0.05876639828690128, 0.9565465190564969]]
[[0.0, 6.938893903907228e-18, 1.0]]
2018-11-20 21:25:00+00:00
1542749100.0
psi= -0.2901445844736547
in:  GSE  out:  MAG
[[0. 0. 1.]]
[[0.1579242866530239, -0.24510951953368185, 0.9565465190564969]]
[[5.551115123125783e-17, -6.938893903907228e-17, 0.9999999999999999]]
2018-11-20 21:25:00+00:00
1542749100.0
psi= -0.2901445844736547
in:  GSE  out:  GEO
[[0. 0. 1.]]
[[-0.1431364038714405, -0.37108394162844693, 0.9175013232427773]]
[[-5.551115123125783e-17, -4.85722573273506e-17, 1.0]]
2018-11-20 21:25:00+00:00
1542749100.0
psi= -0.2901445844736547
in:  GSE  out:  GEI
[[0. 0. 1.]]
[[-5.5